# 4 different ways to Combine two (or more) strategies to backtest on multiple assets 

Reference 0: https://github.com/polakowo/vectorbt/blob/master/tests/notebooks/indicators.ipynb

Reference 1: strategy combination: https://github.com/sergio12S/youtoube/blob/master/vectorbt/combination_parameters.ipynb

Reference 2 - create a custom indicator: https://greyhoundanalytics.com/blog/create-a-custom-indicator-in-vectorbt/

**Description:**

- Assets: Bitcoin, Etheruem
- Strategy used: Dual moving average crossover (DMAC) + Relative Strength Index (RSI)
- Entry: SMA 180's value above SMA 240 & RSI < 80
- Exit: SMA 180's value below SMA 240 & RSI > 20
- Direction = Long Only

### Get the data

In [1]:
import pandas as pd
import numpy
import vectorbt as vbt
from numba import njit

In [2]:
symbols = ["BTC-USD","ETH-USD"]
fast_window, slow_window, rsi_window, top, bottom = (180,240, 14, 80,20)
fees = 0.001
interval = "1D"
start = "2020-01-01"

In [3]:
data = vbt.YFData.download(symbols,
                           start=start, 
                           interval=interval).concat()
data["Close"]

symbol,BTC-USD,ETH-USD
Date,,
2019-12-31 00:00:00+00:00,7193.599121,129.610855
2020-01-01 00:00:00+00:00,7200.174316,130.802002
2020-01-02 00:00:00+00:00,6985.470215,127.410179
2020-01-03 00:00:00+00:00,7344.884277,134.171707
2020-01-04 00:00:00+00:00,7410.656738,135.069366
...,...,...
2023-08-22 00:00:00+00:00,26031.656250,1633.892578
2023-08-23 00:00:00+00:00,26431.640625,1679.274414
2023-08-24 00:00:00+00:00,26162.373047,1659.944580


### Approach 1: using "&" operator

(Note: my preferred method)

In [4]:
rsi = vbt.RSI.run(data["Close"], window=rsi_window, short_name ="RSI")
fast = vbt.MA.run(data["Close"], window=fast_window, short_name="fast")
slow = vbt.MA.run(data["Close"], window=slow_window, short_name="slow")

entries = fast.ma_above(slow).vbt & rsi.rsi_below(top).vbt
exits = fast.ma_below(slow).vbt & rsi.rsi_above(bottom).vbt

pf = vbt.Portfolio.from_signals(data["Close"], entries, exits, fees=fees, freq=interval)
pf.total_return()

fast_window  slow_window  RSI_window  symbol 
180          240          14          BTC-USD    2.243247
                                      ETH-USD    5.198917
Name: total_return, dtype: float64

### Approach 2: Combine indicators using to_numpy()

- (a) built-in vbt indicator + numpy 

In [5]:
def get_signals(close, fast_window, slow_window, rsi_window, top, bottom):
    rsi = vbt.RSI.run(close, window=rsi_window, short_name = "RSI").rsi.to_numpy()
    fast = vbt.MA.run(close, window=fast_window, short_name="fast").ma.to_numpy()
    slow = vbt.MA.run(close, window=slow_window, short_name="slow").ma.to_numpy()
    entries = (fast > slow) & (rsi < top)
    exits = (fast < slow) & (rsi > bottom)
    return entries, exits

entries, exits = get_signals(data["Close"], fast_window, slow_window, 
                             rsi_window, top, bottom)
pf = vbt.Portfolio.from_signals(data["Close"], entries, exits, 
                                fees=fees, freq=interval)
pf.total_return()

symbol
BTC-USD    2.243247
ETH-USD    5.198917
Name: total_return, dtype: float64

- (b) talib indicator + numpy

(Note: got accuracy problem for talib?)

In [6]:
# I try to replicate the above result with talib package but I get inconsistent result for ETH-USD
# (e.g., ETH-USD should be -0.262052 but in this example, I get -0.242040 instead)
## Possible Reason: talib RSI seems calculated in a very different way compared to others: https://github.com/TA-Lib/ta-lib-python/issues/448

def get_signals(close, fast_window, slow_window, rsi_window, top, bottom):
    rsi = vbt.talib("RSI").run(close, timeperiod=rsi_window, short_name = "RSI").real.to_numpy() # talib got problem calculating RSI?
    fast = vbt.talib("MA").run(close, timeperiod=fast_window, short_name="fast").real.to_numpy()
    slow = vbt.talib("MA").run(close, timeperiod=slow_window, short_name="slow").real.to_numpy()
    entries = (fast > slow) & (rsi < top)
    exits = (fast < slow) & (rsi > bottom)
    return entries, exits

entries, exits = get_signals(data["Close"], fast_window, slow_window, 
                             rsi_window, top, bottom)
pf = vbt.Portfolio.from_signals(data["Close"], 
                                entries, exits, 
                                fees=fees, freq=interval)
pf.total_return()

symbol
BTC-USD    2.243247
ETH-USD    5.367018
Name: total_return, dtype: float64

### Approach 4: Combine indicators using vbt.IndicatorFactory

- Reference 1: https://vectorbt.dev/api/indicators/factory/#inputs
- Reference 2: https://greyhoundanalytics.com/blog/create-a-custom-indicator-in-vectorbt/
- Reference 3: https://www.youtube.com/watch?v=57hsQz70vVE&t=209s & https://github.com/mikolaje/TradingTutorial/blob/main/Vector

In [7]:
## create simple moving average strategy
@njit
def ma_indicator(close, window):
    output = numpy.full(close.shape, numpy.nan, dtype=numpy.float_)
    for col in range(close.shape[1]):
        for i in range(window-1, close.shape[0]):
            mavg = numpy.mean(close[i-window+1:i+1, col])
            output[i, col] = mavg
    return output

SMA_factory = vbt.IndicatorFactory(
    input_names=['close'],
    param_names=['window'],
    output_names=['output']
).from_apply_func(ma_indicator)

FAST_SMA, SLOW_SMA =  SMA_factory.run_combs(
    data["Close"],
    window=[fast_window, slow_window],
)
FAST_SMA.output

custom_1_window                     180             
symbol                          BTC-USD      ETH-USD
Date                                                
2019-12-31 00:00:00+00:00           NaN          NaN
2020-01-01 00:00:00+00:00           NaN          NaN
2020-01-02 00:00:00+00:00           NaN          NaN
2020-01-03 00:00:00+00:00           NaN          NaN
2020-01-04 00:00:00+00:00           NaN          NaN
...                                 ...          ...
2023-08-22 00:00:00+00:00  27798.592253  1819.084241
2023-08-23 00:00:00+00:00  27816.556217  1819.478137
2023-08-24 00:00:00+00:00  27833.150651  1819.839412
2023-08-25 00:00:00+00:00  27846.964290  1819.906734
2023-08-26 00:00:00+00:00  27860.992513  1820.005574

[1335 rows x 2 columns]

In [13]:
# create rsi indicator
@njit
def rsi_indicator(close, window):
    # rsi strategy
    delta = close[1:] - close[:-1] # calculate price change over time
    gain = numpy.where(delta>0, delta, 0) 
    loss = -numpy.where(delta<0, delta, 0)
    avg_gain_ = numpy.full(close.shape, numpy.nan, dtype=numpy.float_)
    avg_loss_ = numpy.full(close.shape, numpy.nan, dtype=numpy.float_)
    for col in range(close.shape[1]):
        for i in range(window+1, close.shape[0]):
            avg_gain = numpy.mean(gain[i-window:i, col])
            avg_loss = numpy.mean(loss[i-window:i, col])
            avg_gain_[i, col] = avg_gain
            avg_loss_[i, col] = avg_loss
    rs = numpy.divide(avg_gain_,avg_loss_)
    output = 100 - (100/(1+rs)) # rsi
    return output

RSI_factory = vbt.IndicatorFactory(
    input_names=['close'],
    param_names=['window'],
    output_names=['output']
).from_apply_func(rsi_indicator)

RSI =  RSI_factory.run(
    data["Close"],
    window=rsi_window,
)
RSI.output

custom_window                     14           
symbol                       BTC-USD    ETH-USD
Date                                           
2019-12-31 00:00:00+00:00        NaN        NaN
2020-01-01 00:00:00+00:00        NaN        NaN
2020-01-02 00:00:00+00:00        NaN        NaN
2020-01-03 00:00:00+00:00        NaN        NaN
2020-01-04 00:00:00+00:00        NaN        NaN
...                              ...        ...
2023-08-22 00:00:00+00:00   6.597931  10.802279
2023-08-23 00:00:00+00:00  15.204020  23.233388
2023-08-24 00:00:00+00:00  14.753407  22.163288
2023-08-25 00:00:00+00:00  14.494379  21.946816
2023-08-26 00:00:00+00:00  14.167491  21.495593

[1335 rows x 2 columns]

In [9]:
entries = (FAST_SMA.output_above(SLOW_SMA).vbt & RSI.output_below(top).vbt)
exits = (FAST_SMA.output_below(SLOW_SMA).vbt & RSI.output_above(bottom).vbt)

pf = vbt.Portfolio.from_signals(data["Close"],
                                entries, exits,
                                fees=fees, freq=interval)
pf.total_return()

custom_1_window  custom_2_window  custom_window  symbol 
180              240              14             BTC-USD    2.243247
                                                 ETH-USD    5.198917
Name: total_return, dtype: float64